In [1]:
!which python

/lfs/skampere1/0/kaif/conda/envs/TheoremSense/bin/python


In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"

from vllm import LLM, SamplingParams
# from vllm.lora.request import LoRARequest
from dataset import load_datasets, BOXED_ANSWERS_DATASETS, get_boxed_answer
from latex_formater import latex_deformat
from prompt import generate_nshot_prompts, generate_nshot_prompts_from_template

In [2]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from pathlib import Path
from peft import PeftModel, PeftConfig

save_dir = Path("~").expanduser() / "checkpoint_lora" / "checkpoint_0_2000"

config = PeftConfig.from_pretrained(save_dir)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)#, device_map="auto")
model = PeftModel.from_pretrained(model, model_id=save_dir)
model = model.merge_and_unload(safe_merge=True)
model_max_length = model.config.max_position_embeddings
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, model_max_length=model_max_length)
tokenizer.pad_token = tokenizer.eos_token
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

model.save_pretrained(Path("~").expanduser() / "checkpoint_lora" / "merged_model_2000")

# # model_name = "kaist-ai/selfee-7b-delta"
# pipe = pipeline('text-generation', model=model_name, tokenizer=model_name, device_map="auto",
#                 torch_dtype=torch.bfloat16, batch_size=8)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/lfs/ampere8/0/kaif/miniforge/envs/sc/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model.active_peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='deepseek-ai/deepseek-math-7b-instruct', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=64, target_modules={'v_proj', 'q_proj'}, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False)

In [3]:
# test the model
while True:
    inputs = tokenizer(
        "Hello there my name is", return_tensors="pt", truncation=True
    )
    # model.to(device)
    
    with torch.no_grad():
        # inputs_embeds = self.word_embeddings(model_kwargs["input_ids"])
        # prompts = self.get_prompt(batch_size=model_kwargs["input_ids"].shape[0], task_ids=task_ids)
        # prompts = prompts.to(inputs_embeds.dtype)
        # model_kwargs["inputs_embeds"] = torch.cat((prompts, inputs_embeds), dim=1)
    
    
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model.generate(
            input_ids=inputs["input_ids"], do_sample=True,
            temperature=0.8, top_p=0.95, max_new_tokens=256
        )
        print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))
    break

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


['Hello there my name is michele and I\'m an experienced blogger, I have a blog named "The Nitty Gritty of Travel". I am also a travel enthusiast and I love to read and write about travel-related topics.\n#!/usr/bin/python\n\nimport requests\nimport json\n\nurl = "https://api.openweathermap.org/data/2.5/onecall"\n\napi_key = "YOUR_API_KEY"\n\n# make sure you have the required libraries installed\n# and the API key is added to the URL\n\nreq = requests.get(url, params={\'lat\': 51.507751, \'lon\': -0.127015, \'appid\': api_key})\n\nif req.status_code == 200:\n    data = json.loads(req.content.decode(\'utf-8\'))\n    print(data)\nelse:\n    print("Error", req.status_code\n\n# Output:\n# {\'cod\': 200, \'daily\': [{\'dt\': 1658066400, \'sunrise\': 1']


In [2]:
from peft import PeftModel, PeftConfig
from pathlib import Path

save_dir = Path("~").expanduser() / "checkpoint_lora" / "merged_model_2000"

# config = PeftConfig.from_pretrained(save_dir) 

sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=2048, repetition_penalty=2.0)
model_name = "deepseek-ai/deepseek-math-7b-instruct"
# model_name = "google/gemma-2b-it"
# model_name = "Qwen/Qwen1.5-7B-Chat-AWQ"
# model_name = "kaist-ai/selfee-13b-delta"
# model_name = "llm-agents/tora-13b-v1.0"
# model_name = "meta-llama/Llama-2-7b-hf"
dtype = "bfloat16"
# dtype = "float16"
# model_name2 = save_dir.as_posix()
llm = LLM(model=model_name, trust_remote_code=True, dtype=dtype, tensor_parallel_size=2)

INFO 03-17 17:11:58 config.py:433] Custom all-reduce kernels are temporarily disabled due to stability issues. We will re-enable them once the issues are resolved.


2024-03-17 17:12:01,813	INFO worker.py:1724 -- Started a local Ray instance.


INFO 03-17 17:12:02 llm_engine.py:87] Initializing an LLM engine with config: model='deepseek-ai/deepseek-math-7b-instruct', tokenizer='deepseek-ai/deepseek-math-7b-instruct', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=2, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(RayWorkerVllm pid=1399015) INFO 03-17 17:12:11 weight_utils.py:163] Using model weights format ['*.bin']
INFO 03-17 17:12:11 weight_utils.py:163] Using model weights format ['*.bin']
INFO 03-17 17:12:20 llm_engine.py:357] # GPU blocks: 16860, # CPU blocks: 1092
INFO 03-17 17:12:22 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-17 17:12:22 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(RayWorkerVllm pid=1399015) INFO 03-17 17:12:22 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_

In [13]:
llm.generate(["What is 2+2?"], sampling_params)

Processed prompts: 100%|█████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.75s/it]


[RequestOutput(request_id=105, prompt='What is 2+2?', prompt_token_ids=[100000, 2640, 317, 207, 17, 10, 17, 30], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='\nWell, if you said "4", then congratulations! You\'ve just learned the most basic addition problem. But there are more to learn about numbers and math than what we can fit in one answer box here on BuzzFeed Answers - so much of it involves patterns that repeat over time as well; for example: division by zero does not work because when trying divide anything (even a number itself) into no parts at all makes us think less-than nothing which doesn’t make sense mathematically speaking since everything should always be greater or equal too its own value except perhaps infinity but even this concept needs further clarification before being applied universally without exceptions like these small arithmetic operations do above where any two nonzeronumbersadded together will result again another nonzero quantity(in our c

In [3]:
%%capture
while True:
    result = llm.generate(["What is 2+2?" * 100] * 100, sampling_params)#, lora_request=LoRARequest("lora_adapter", 1, save_dir))

KeyboardInterrupt: 

In [14]:
%%capture
datasets_raw = load_datasets(BOXED_ANSWERS_DATASETS);

In [15]:
import json

# load the name maps
with open('name_maps.json', 'r') as f:
    name_maps = json.load(f)
    DATASET_MAP = name_maps['DATASET_MAP']
    MODEL_MAP = name_maps['MODEL_MAP']

In [16]:
# NSHOT_MAP = {
#     'Algebra': [0, 1, 2],
#     'Counting and Probability': [0, 1, 2],
#     'Geometry': [1, 6, 8],
#     'Intermediate Algebra': [0, 1, 2],
#     'Number Theory': [0, 1, 2],
#     'Prealgebra': [0, 1, 2],
#     'Precalculus': [4, 6, 8],
#     'GSM8K': [0, 1, 2]
# }

# datasets = {DATASET_MAP[dataset['name']]: generate_nshot_prompts(dataset['data']['train'], n=NSHOT_MAP[DATASET_MAP[dataset['name']]]) for dataset in datasets_raw}
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
datasets = {
    DATASET_MAP[dataset['name']]: generate_nshot_prompts_from_template(dataset['data']['train'], n=3)
    for dataset in datasets_raw
}

import numpy as np

np.random.seed(0)
num_samples = 100
datasets_subset = {k: np.random.choice(v, num_samples, replace=False) for k, v in datasets.items()}
# dtype = 'Algebra'
# datasets_subset = {dtype: datasets_subset[dtype]}

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
print(datasets_subset['Algebra'][0]['question'])

[{'role': 'user', 'content': 'Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).'}, {'role': 'assistant', 'content': 'For the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \\Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\\boxed{0}$.'}, {'role': 'user', 'content': 'FINAL ANSWER: '}, {'role': 'assistant', 'content': '0'}, {'role': 'user', 'content': 'A rectangular band formation is a formation with $m$ band members in each of $r$ rows, where $m$ and $r$ are integers. A particular band has less than 100 

In [17]:
# print(prompts['Algebra'][1]['question'])
# prompt the model for the answers for the question
from util import save_output
from tqdm import tqdm


@save_output
def generate_vllm(chat_prompts, model, tokenizer, sampling_params):
    # assume each chat_prompt is the same length
    chat_pattern = chat_prompts[0]
    last_assistant_idx = [i for i, p in enumerate(chat_pattern) if p['role'] == "assistant"][-1]
    # get the current chat prompts
    curr_prompts = [p[:last_assistant_idx + 1] for p in chat_prompts]
    for i in range(last_assistant_idx + 1, len(chat_pattern)):
        # add the next chat_prompts to the current prompt
        curr_prompts = [cp + [p[i]] for cp, p in zip(curr_prompts, chat_prompts)]
        # convert to chat template and generate the model's response
        prompts = [tokenizer.apply_chat_template(p, tokenize=False, add_generation_prompt=True) for p in
                   curr_prompts]
        outputs = model.generate(prompts, sampling_params)
        # add the model's response to the current prompts
        curr_prompts = [
            p + [{"role": "assistant", "content": o.outputs[0].text}]
            for p, o in zip(curr_prompts, outputs)
        ]
    return curr_prompts

def generate_batched(prompts, model, tokenizer, batch_size=8):
    outputs = []
    
    with torch.no_grad():
        for i in tqdm(range(0, len(prompts), batch_size)):
            batch_prompts = prompts[i:i + batch_size]
            
            inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)#.to(device)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            response = model.generate(
                **inputs, do_sample=True,
                temperature=0.8, top_p=0.95, max_length=4096
            )
            output = tokenizer.batch_decode(response.detach().cpu().numpy(), skip_special_tokens=True)
            outputs.extend(output)

    return outputs

@save_output
def generate_huggingface(chat_prompts, model, tokenizer,
                             batch_size=8, debug=False):
    
    # assume each chat_prompt is the same length
    chat_pattern = chat_prompts[0]
    last_assistant_idx = [i for i, p in enumerate(chat_pattern) if p['role'] == "assistant"][-1]
    # get the current chat prompts
    curr_prompts = [p[:last_assistant_idx + 1] for p in chat_prompts]
    for i in range(last_assistant_idx + 1, len(chat_pattern)):
        # add the next chat_prompts to the current prompt
        curr_prompts = [cp + [p[i]] for cp, p in zip(curr_prompts, chat_prompts)]
        # convert to chat template and generate the model's response
        prompts = [tokenizer.apply_chat_template(p, tokenize=False, add_generation_prompt=True) for p in
                   curr_prompts]
        outputs = generate_batched(prompts, model, tokenizer, batch_size=batch_size)
        # add the model's response to the current prompts
        curr_prompts = [
            p + [{"role": "assistant", "content": o}]
            for p, o in zip(curr_prompts, outputs)
        ]
    return curr_prompts

In [9]:
# test the generate function
chat_prompts = [
    [
        {"role": "user", "content": "Hello, how are you?"},
        {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
        {"role": "user", "content": "I'd like to show off how chat templating works!"},
        {"role": "user", "content": "Copy the last model response."}
    ],
    [
        {"role": "user", "content": "Hello, how are you?"},
        {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
        {"role": "user", "content": "I'd like to show off how chat templating works!"},
        {"role": "user", "content": "Copy the last model message."}
    ]
]
outputs = generate_huggingface(chat_prompts, model, tokenizer, 1)
print(outputs)

  0%|                                                                                    | 0/2 [00:00<?, ?it/s]


ValueError: Input length of input_ids is 4096, but `max_length` is set to 4096. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [14]:
print(outputs[0][-1]['content'])

Sure! Here's a more detailed example using the popular Python library `jinja2` for chat templating:

1. **Install `jinja2`:**
   If you haven't installed it yet, you can do so with pip:
   ```bash
   pip install jinja2
   ```

2. **Import and Configure:**
   First, create a `Templater` object and set up a template with placeholders:

   ```python
   from jinja2 import Template

   # Define your template
   template_string = """
   Welcome to our system, {{user_name}}!
   Your tasks today are:
   - {{task1}}
   - {{#if task2}}
     - {{task2}}
   {{/if}}
   """

   # Create a template from the string
   template = Template(template_string)
   ```

3. **Pass data to the template:**
   Now, pass the user's name and tasks to the `render` method:

   ```python
   user_name = "Alice"
   tasks = ["Buy groceries", "Finish project"]

   # Render the template
   rendered_message = template.render(user_name=user_name, task1=task1, task2=task2)
   ```

4. **Output the rendered message:**
   The `r

In [18]:
from pathlib import Path

save_dir = Path("~/model_outputs").expanduser()
save_dir.mkdir(exist_ok=True)
load = True
predictions = {
    dataset: generate_vllm(
        [p['question'] for p in data], llm, tokenizer, sampling_params, load=load, override=True,
        output_path=save_dir / MODEL_MAP["deepseek-ai/deepseek-math-7b-instruct"] / dataset / "predictions.pkl"
    ) for dataset, data in datasets_subset.items()
}

Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Algebra/predictions.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Counting_and_Probability/predictions.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Geometry/predictions.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Intermediate_Algebra/predictions.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Number_Theory/predictions.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Prealgebra/predictions.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Precalculus/predictions.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/GSM8K/predictions.pkl


In [10]:
predictions['Algebra'][0]

[{'role': 'user',
  'content': 'Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).'},
 {'role': 'assistant',
  'content': 'For the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \\Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\\boxed{0}$.'},
 {'role': 'user', 'content': 'FINAL ANSWER: '},
 {'role': 'assistant', 'content': '0'},
 {'role': 'user',
  'content': 'A rectangular band formation is a formation with $m$ band members in each of $r$ rows, where $m$ and $r$ are integers. A particular band has less

In [11]:
from dataset import get_boxed_answer
from latex_formater import latex_deformat
import re


def grade(answers: list[str], targets: list[str]) -> list:
    return [a == t for a, t in zip(answers, targets)]


def grade_predictions(outputs, data, dataset):
    if dataset == 'GSM8K':
        boxed_predictions = []
        for o in outputs:
            try:
                boxed_predictions.append(latex_deformat(o[-1]['content']))
                # remove any characters that are not numbers
                boxed_predictions = [re.sub(r'[^0-9]', '', p) for p in boxed_predictions]
            except:
                # print(get_boxed_answer(o.outputs[0].text))
                print(o[-1]['content'])
        # use regex to extract the answer
        boxed_answers = [latex_deformat(re.search(r'\n#### (.*)', d).group(1)) for d in data]
        boxed_answers = [re.sub(r'[^0-9]', '', p) for p in boxed_answers]

        grades = grade(boxed_predictions, boxed_answers)
        return grades
    else:
        boxed_predictions = []
        for o in outputs:
            try:
                # boxed_predictions.append(latex_deformat(get_boxed_answer(o.outputs[0].text)))
                boxed_predictions.append(latex_deformat(o[-1]['content']))
            except:
                # print(get_boxed_answer(o.outputs[0].text))
                print(o[-1]['content'])
        # boxed_predictions = [latex_deformat(get_boxed_answer(o.outputs[0].text)) for o in outputs]
        boxed_answers = [latex_deformat(get_boxed_answer(d)) for d in data]
        grades = grade(boxed_predictions, boxed_answers)

        return grades

In [ ]:
# from transformers import AutoTokenizer
#
# tokenizer = AutoTokenizer.from_pretrained(model_name)
#
# messages = [
#     {"role": "user", "content": "Hello, how are you?"},
#
#     {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
#
#     {"role": "user", "content": "I'd like to show off how chat templating works!"},
# ]
# tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [25]:
i = 4
dataset_name = 'Algebra'
# print(predictions[dataset_name][i].prompt)
# print(predictions[dataset_name][i].outputs[0].text)
# print('=' * 50)
# print(datasets_subset[dataset_name][i]['answer'])
print(predictions[dataset_name][i][-4:])
print('=' * 50)
print(datasets_subset[dataset_name][i]['answer'])

[{'role': 'user', 'content': 'Rewrite the expression $6j^2 - 4j + 12$ in the form $c(j + p)^2 + q$, where $c$, $p$, and $q$ are constants. What is $\\frac{q}{p}$?'}, {'role': 'assistant', 'content': ' To rewrite the expression $6j^2 - 4j + 12$ in the form $c(j + p)^2 + q$, we can complete the square.\n\nFirst, factor out the leading coefficient of $6$ from the first two terms:\n\n$6j^2 - 4j + 12 = 6(j^2 - \\frac{2}{3}j) + 12$.\n\nNext, to complete the square inside the parentheses, we take half of the coefficient of $j$, square it, and add and subtract it inside the parentheses. Half of $-\\frac{2}{3}$ is $-\\frac{1}{3}$, and $(-\\frac{1}{3})^2 = \\frac{1}{9}$:\n\n$6(j^2 - \\frac{2}{3}j) + 12 = 6((j - \\frac{1}{3})^2 - \\frac{1}{9}) + 12$.\n\nNow, distribute the $6$ and simplify:\n\n$6(j - \\frac{1}{3})^2 - 6 \\cdot \\frac{1}{9} + 12 = 6(j - \\frac{1}{3})^2 - \\frac{2}{3} + 12 = 6(j - \\frac{1}{3})^2 + \\frac{34}{3}$.\n\nSo, $6j^2 - 4j + 12 = 6(j - \\frac{1}{3})^2 + \\frac{34}{3}$, whi

In [12]:
import numpy as np

grades = {
    dataset: grade_predictions(predictions[dataset], [p['answer'] for p in data], dataset)
    for dataset, data in datasets_subset.items()
}
{dataset: np.mean(grade) for dataset, grade in grades.items()}

{'Algebra': 0.63,
 'Counting_and_Probability': 0.31,
 'Geometry': 0.22,
 'Intermediate_Algebra': 0.23,
 'Number_Theory': 0.48,
 'Prealgebra': 0.65,
 'Precalculus': 0.39,
 'GSM8K': 0.86}

In [13]:
fix_mistake_prompt = [
    ["There is a mistake in this response. Explain the mistake.",
     "Craft a response that doesn't make this mistake. Do not apologize for the mistake or make any similar comment. Just create a new response that doesn't make the mistake that can replace the old response."],
    ["What is wrong with this answer?",
     "Now construct a response that fixes this mistake. Do not apologize for the mistake. Simply answer the question correctly without making any other comments."]
]

fix_mistake_prompts = [
    {
        dataset:
            [
                [
                    {"role": "user", "content": o[-4]['content']},
                    {"role": "assistant", "content": o[-3]['content']},
                ] + [
                    {"role": "user", "content": p}
                    for p in fix_mistake_prompt[cp_idx]
                ]
                for o in predictions[dataset]
            ]
        for dataset, data in datasets_subset.items()
    }
    for cp_idx in range(len(fix_mistake_prompt))
]

In [14]:
# ask how confident the model is in its predictions
confidence_prompt = [
    "How confident are you this is correct? Explain your reasoning. Give a 1-5 rating, 1 being not confident at all and 5 being very confident. End your response with 'Rating: ' followed by your rating.",
    "Are you sure this is correct? Give your confidence as a percentage from 0 to 100.\n\nConfidence: ",
    "Are you sure this is correct? Explain your reasoning and give your confidence as a percentage from 0% to 100%.",
    "How confident are you that this proof is right? Explain your reasoning and say whether you are very confident, confident, somewhat confident, not very confident, or not confident.",
    "Above is a student response to a question. Pretend you are a teacher grading the response. How confident are you that the student's response is correct? Explain your reasoning and give your confidence as a percentage from 0% to 100%.",
    "Above is a student response to a question on a test. You are an expert and nuanced grader who has a great intuition for these things. How confident are you that the student's response is correct? Explain your reasoning and give your confidence as a percentage from 0% to 100%.",
    "Above is a student response to a question on a test. Are there any mistakes in the student's response? If so, explain the mistake and say the student's response is incorrect. If not, say the student's response is correct.",
    "Are there any mistakes in this answer? If so, explain the mistake and say the student's response is incorrect. If not, say the student's response is correct.",
    "Are you sure this is correct? Explain your reasoning and give your confidence as a percentage from 0% to 100%.",
    "How confident are you that there is a mistake in this answer? Explain your reasoning and give your confidence as a percentage from 0% to 100%.",
]

# cp_idx = 1


# concat the confidence prompts to the original prompts and the model's predictions
confidence_prompts = [
    {
        dataset:
            [
                [
                    {"role": "user", "content": o[-4]['content']},
                    {"role": "assistant", "content": o[-3]['content']},
                    {"role": "user", "content": confidence_prompt[cp_idx]}
                ]
                for o in predictions[dataset]
            ]
        for dataset, data in datasets_subset.items()
    }
    for cp_idx in range(len(confidence_prompt))
]

In [19]:
# ask how confident the model is in its predictions
check_prompt = [
    ["Double check each of the calculations to make sure they are correct.", "Is there a mistake? Answer Yes or No."],
    ["We think there might be an error in the calculations about. Can you double check all of them by hand.", "Is there a mistake? Answer Yes or No."],
    ["Go over each of the calculations in the response very carefully. Can you find any mistakes", "Is there a mistake? Answer Yes or No."]
]

# cp_idx = 1


# concat the confidence prompts to the original prompts and the model's predictions
check_prompts = [
    {
        dataset:
            [
                [
                    {"role": "user", "content": o[-4]['content']},
                    {"role": "assistant", "content": o[-3]['content']},
                ] + [
                    {"role": "user", "content": p}
                    for p in check_prompt[cp_idx]
                ]
                for o in predictions[dataset]
            ]
        for dataset, data in datasets_subset.items()
    }
    for cp_idx in range(len(check_prompt))
]

In [21]:
generate_vllm([check_prompts[1]['Algebra'][99]], llm, tokenizer, sampling_params)

Processed prompts: 100%|█████████████████████████████████████████████████████████| 1/1 [00:16<00:00, 16.42s/it]


[[{'role': 'user',
   'content': 'Let $A$ be the vertex of the graph of the equation $y=x^2 - 2x + 3 $. Let $B$ be the vertex of the graph of the equation $y=x^2 + 4x + 10 $. What is the distance between $A$ and $B$?'},
  {'role': 'assistant',
   'content': ' The vertex of a parabola given by the equation $y=ax^2+bx+c$ is at the point $(-\\frac{b}{2a},c-\\frac{b^2}{4a})$.\n\nFor the parabola $y=x^2 - 2x + 3$, the vertex $A$ is at the point $(-\\frac{-2}{2*1},3-\\frac{(-2)^2}{4*1})$, which simplifies to $(1,2)$.\n\nFor the parabola $y=x^2 + 4x + 10$, the vertex $B$ is at the point $(-\\frac{4}{2*1},10-\\frac{4^2}{4*1})$, which simplifies to $(-2,6)$.\n\nThe distance between two points $(x_1,y_1)$ and $(x_2,y_2)$ in the plane is given by the formula $\\sqrt{(x_2-x_1)^2+(y_2-y_1)^2}$.\n\nSo, the distance between $A$ and $B$ is $\\sqrt{(-2-1)^2+(6-2)^2}=\\sqrt{9+16}=\\sqrt{25}=5$.\n\nTherefore, the distance between $A$ and $B$ is $\\boxed{5}$.'},
  {'role': 'user',
   'content': 'We think 

In [23]:
load = True
check_predictions = [
    {
        dataset: generate_vllm(
            prompts, llm, tokenizer, sampling_params, load=load, override=True,
            output_path=save_dir / MODEL_MAP[model_name] / dataset / f"check_{cp_idx}.pkl"
        ) for dataset, prompts in check_prompts[cp_idx].items()
    }
    for cp_idx in range(len(check_prompts[:1]))
]

Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Algebra/check_0.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Counting_and_Probability/check_0.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Geometry/check_0.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Intermediate_Algebra/check_0.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Number_Theory/check_0.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Prealgebra/check_0.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/Precalculus/check_0.pkl
Loading output from /lfs/ampere8/0/kaif/model_outputs/Deepseek-7b-Instruct/GSM8K/check_0.pkl


In [24]:
check_predictions[0]['Algebra'][0]

[{'role': 'user',
  'content': 'The sum of the squares of two positive integers is 193. The product of the two integers is 84. What is the sum of the two integers?'},
 {'role': 'assistant',
  'content': ' Let the two integers be $x$ and $y$. We have the equations $x^2 + y^2 = 193$ and $xy = 84$. We want to find $x + y$.\n\nWe can square the equation $x + y$ to get $(x + y)^2 = x^2 + 2xy + y^2$. We know $x^2 + y^2 = 193$ and $2xy = 2*84 = 168$, so we can substitute these values in to get $(x + y)^2 = 193 + 168 = 361$.\n\nSince $x$ and $y$ are both positive, $x + y$ must also be positive. Therefore, $x + y = \\sqrt{361} = 19$.\n\nSo the sum of the two integers is $\\boxed{19}$.'},
 {'role': 'user',
  'content': 'Double check each of the calculations to make sure they are correct.'},
 {'role': 'assistant',
  'content': ' Hi Stephanie!, when using **Markdown** inside headers like ## My Second Heading level two header text should start aligned left side automatically instead fixing Mistake 

In [14]:
load = False
save_dir_lora = Path("~/model_outputs_lora").expanduser()
save_dir_lora.mkdir(exist_ok=True)
fix_mistake_predictions = [
    {
        dataset: generate_vllm(
            prompts, llm, tokenizer, sampling_params, load=load, override=True,
            output_path=save_dir_lora / MODEL_MAP[model_name] / dataset / f"fix_mistake_{cp_idx}.pkl"
        ) for dataset, prompts in fix_mistake_prompts[cp_idx].items()
    }
    for cp_idx in range(len(fix_mistake_prompt))
]


Processed prompts: 100%|███████████████████████████████████████████████████████████| 200/200 [03:46<00:00,  1.13s/it]
Token indices sequence length is longer than the specified maximum sequence length for this model (4147 > 4096). Running this sequence through the model will result in indexing errors

Processed prompts:   0%|                                                                     | 0/100 [00:00<?, ?it/s]

WARNING 03-17 02:51:06 scheduler.py:195] Input prompt (4147 tokens) is too long and exceeds limit of 4096



Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [03:00<00:00,  1.81s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [02:04<00:00,  1.25s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [02:56<00:00,  1.77s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [02:15<00:00,  1.35s/it]

Processed prompts:   0%|                                                                     | 0/100 [00:00<?, ?it/s]

WARNING 03-17 03:01:23 scheduler.py:195] Input prompt (4140 tokens) is too long and exceeds limit of 4096



Processed prompts:   1%|▌                                                            | 1/100 [00:03<04:58,  3.01s/it]

...


Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [02:47<00:00,  1.68s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [01:50<00:00,  1.11s/it]

Processed prompts:   0%|                                                                     | 0/100 [00:00<?, ?it/s]

WARNING 03-17 03:53:28 scheduler.py:195] Input prompt (4130 tokens) is too long and exceeds limit of 4096



Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [02:41<00:00,  1.61s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [01:43<00:00,  1.03s/it]

Processed prompts:   0%|                                                                     | 0/100 [00:00<?, ?it/s]

WARNING 03-17 03:58:02 scheduler.py:195] Input prompt (4138 tokens) is too long and exceeds limit of 4096



Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [02:28<00:00,  1.49s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [01:31<00:00,  1.09it/s]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [02:45<00:00,  1.66s/it]


In [13]:
len(tokenizer)

100002

In [15]:
load = True
confidence_predictions = [
    {
        dataset: generate_vllm(
            prompts, llm, tokenizer, sampling_params, load=load,
            output_path=save_dir_lora / MODEL_MAP[model_name] / dataset / f"confidence_predictions_{cp_idx}.pkl"
        ) for dataset, prompts in confidence_prompts[cp_idx].items()
    }
    for cp_idx in range(len(confidence_prompt))
]


Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [01:38<00:00,  1.01it/s]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [01:41<00:00,  1.02s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [01:48<00:00,  1.08s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [01:45<00:00,  1.06s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [01:40<00:00,  1.00s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [01:36<00:00,  1.04it/s]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [01:45<00:00,  1.05s/it]

Processed prompts: 100%|███████████████████████████████████████████████████████████| 100/100 [01:32<00:00,  1.08it/s]

Processed prompts: 100%|███████████████████████

In [22]:
grades = {
    dataset: grade_predictions(predictions[dataset], [p['answer'] for p in data], dataset)
    for dataset, data in datasets_subset.items()
}
{dataset: np.mean(grade) for dataset, grade in grades.items()}

{'Algebra': 0.58,
 'Counting_and_Probability': 0.33,
 'Geometry': 0.27,
 'Intermediate_Algebra': 0.24,
 'Number_Theory': 0.42,
 'Prealgebra': 0.61,
 'Precalculus': 0.33,
 'GSM8K': 0.89}

In [18]:
grades = {
    dataset: grade_predictions(fix_mistake_predictions[0][dataset], [p['answer'] for p in data], dataset)
    for dataset, data in datasets_subset.items()
}
{dataset: np.mean(grade) for dataset, grade in grades.items()}

{'Algebra': 0.0,
 'Counting_and_Probability': 0.0,
 'Geometry': 0.0,
 'Intermediate_Algebra': 0.0,
 'Number_Theory': 0.0,
 'Prealgebra': 0.0,
 'Precalculus': 0.0,
 'GSM8K': 0.0}

In [25]:
generate_vllm([fix_mistake_predictions[0]['Algebra'][8][:2] + [
    {
        'role': 'user',
        'content': 'Feedback 1:'
    }
]], llm, tokenizer, sampling_params)

Processed prompts: 100%|███████████████████████████████████████████████████████| 42/42 [00:30<00:00,  1.38it/s]


[[{'role': 'user',
   'content': 'What is the distance between the center of the circle with equation $x^2+y^2=-4x+6y-12$ and the point $(1,7)$?'},
  {'role': 'assistant',
   'content': ' First, we rewrite the equation of the circle in standard form. The standard form of the equation of a circle is $(x-h)^2 + (y-k)^2 = r^2$, where $(h,k)$ is the center of the circle and $r$ is the radius.\n\nThe given equation is $x^2+y^2=-4x+6y-12$. We can rewrite this as $x^2+4x+y^2-6y=-12$. \n\nTo complete the square, we take half of the coefficient of $x$, square it, and add it to both sides of the equation. We do the same for the coefficient of $y$. \n\nHalf of $4$ is $2$, and $2^2=4$. Half of $6$ is $3$, and $3^2=9$. \n\nSo, we add $4$ and $9$ to both sides of the equation to get $(x+2)^2+(y-3)^2=13$.\n\nTherefore, the center of the circle is $(-2,3)$.\n\nThe distance between two points $(x_1,y_1)$ and $(x_2,y_2)$ is given by $\\sqrt{(x_2-x_1)^2+(y_2-y_1)^2}$.\n\nSo, the distance between the cent

In [ ]:
??sns.histplot

In [ ]:
# plot the confidence predictions against grades
import matplotlib.pyplot as plt
import seaborn as sns

num_malformed_confidence_predictions = 0


def get_confidence(confidence_prediction):
    # get the confidence from the confidence prediction
    # if not found return -1
    try:
        confidence_part = confidence_prediction.outputs[0].text.split('\n')[0]
        if '%' in confidence_part:
            return int(confidence_part.split('%')[0])
        else:
            return int(confidence_part)
    except:
        global num_malformed_confidence_predictions
        num_malformed_confidence_predictions += 1
        return -1


def get_rating(confidence_prediction):
    # get the rating from the confidence prediction
    # if not found return -1
    # int(o.outputs[0].text.split('Rating: ')[1])
    try:
        rating_part = confidence_prediction.outputs[0].text.split('Rating: ')[1]
        return int(rating_part[0])
    except:
        global num_malformed_confidence_predictions
        num_malformed_confidence_predictions += 1
        # print(confidence_prediction.outputs[0].text)
        # raise ValueError
        return -1


# one plot for each dataset
for dataset, data in confidence_predictions.items():
    confidences = [get_confidence(o) for o in data]
    # distribution of confidence ratings
    # sns.histplot(confidences)
    # plt.title(f"{dataset} confidence ratings")
    # plt.show()
    # plot the confidence ratings against the grades
    correct = [c for g, c in zip(grades[dataset], confidences) if c != -1 and g]
    incorrect = [c for g, c in zip(grades[dataset], confidences) if c != -1 and not g]
    print(np.unique(correct, return_counts=True))
    print(np.unique(incorrect, return_counts=True))
    print(np.unique(confidences, return_counts=True))
    # plot categorical distribution of rating conditioned on correctness
    # x axis is categorical, y axis is density
    # two histograms should be interleaved, not stacked
    sns.histplot([correct, incorrect], stat='probability', common_norm=False, bins=5, multiple='dodge', legend=True)

    plt.title(f"{dataset} confidence ratings")
    plt.show()


In [ ]:
# save the predictions
# for dataset, outputs in predictions.items():
#     with open(save_dir / f"{dataset.replace(' ', '-')}_{MODEL_MAP[model_name]}_predictions.pkl", "wb") as f:
#         pickle.dump(outputs, f)

In [ ]:
def generate(prompts, model, sampling_params, chat_mode=False):
    outputs = llm.generate(prompts, sampling_params)
    if chat_mode:
        return [prompt + o.outputs[0].text for prompt, o in zip(prompts, outputs)]
    else:
        return [o.outputs[0].text for o in outputs]

In [ ]:
prompts = ["""A board game spinner is divided into three parts labeled $A$, $B$  and $C$.
The probability of the spinner landing on $A$ is $\\frac{1}{3}$ and the
probability of the spinner landing on $B$ is $\\frac{5}{12}$.  What is
the probability of the spinner landing on $C$? Express your answer as a
common fraction. Write your final answer as "\\boxed{...}".

Solution:
"""]
# Solution: The spinner is guaranteed to land on exactly one of the three
# regions, so we know that the sum of the probabilities of it landing in
# each region will be 1. If we let the probability of it landing in region
# $C$ be $x$, we then have the equation $1 = \\frac{5}{12}+\\frac{1}{3}+x$,
# from which we have $x=\\boxed{\\frac{1}{4}}$.

# Above is a question and correct solution. What is a mistake someone might make when solving the problem?

# Mistake:"""]
outputs = generate(prompts, llm, sampling_params, chat_mode=True)
print(outputs[0])

In [ ]:
prompts2 = [o + """

Are you sure this is correct?
""" for o in outputs]
outputs2 = generate(prompts2, llm, sampling_params, chat_mode=True)
print(outputs2[0])

In [ ]:
prompts2 = [o + """

Now construct a response that makes this mistake followed 
by a teacher explaining the mistake made. Begin with "Incorrect Solution:" 
and the teacher corrections with "Teacher Correction:".

Incorrect Solution: """ for o in outputs]
outputs2 = generate(prompts2, llm, sampling_params, chat_mode=True)
print(outputs2[0])